In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import scipy.io as spi
%matplotlib inline

Functions for the monomials

In [ ]:
%run monomials

In [ ]:
%run pca

## Get the representers from the FE code

In [ ]:
%run femshape

space = Space(order=4, meshsize=3,L=1)

In [ ]:
%run utils

Test 1: Construct 5 random variations of the circle and a figure of 8

In [ ]:
npoints = 50
ncurves = 10
x = np.zeros((ncurves,npoints))
y = np.zeros((ncurves,npoints))

x[1:,:],y[1:,:] = randshapes(ncurves-1,npoints)

x[0], y[0] = figure_of_eight(npoints)

Using the finite elements

In [ ]:
curves = np.stack([x,y], axis=-1)
reps = [Representer(Current(space, curve)) for curve in curves]

In [ ]:
def pca_data(representers, order=1):
    uv = np.array([rep.get(order=order) for rep in representers])
    pe = uv.reshape(len(representers),-1,order='F')
    return pe

In [ ]:
pe = pca_data(reps, 1)

In [ ]:
pca = PCA(pe, x, y)

In [ ]:
plt.figure(figsize=(8,8))
pca.plot(scaling=1)
plt.axis('equal');

Using the monomials

In [ ]:
cl = (npoints-1)*np.ones((1,ncurves),dtype=int)
u = representer(x,y,ncurves,cl,npoints,10)
u1 = np.zeros((ncurves,np.shape(u)[1]*np.shape(u)[2]*2))
for i in range(ncurves):
    u1[i,:] = np.matrix.flatten(u[i,:,:,:])

In [ ]:
pca = PCA(u1, x, y)

In [ ]:
pca.plot(scaling=.01)

Save a load of data into matlab file for easy comparison

In [ ]:
import scipy.io as spi
ncurves = 200
npoints = 500
x,y = randshapes(ncurves,npoints)
spi.savemat('currents_data_200_500.mat',{'x':x,'y':y})

In [ ]:
def load_data(name, ncurves):
    data = spi.loadmat(name)
    x = data['x']
    y = data['y']
    x = x[:ncurves,:]
    y = y[:ncurves,:]
    return x, y

PCA on the representer space, not Euclidean space

In [ ]:
x, y = load_data('currents_data_200_500.mat', 32)

In [ ]:
curves = np.stack([x,y], axis=-1)
reps = [Representer(Current(space, curve)) for curve in curves]

In [ ]:
pe = pca_data(reps)

In [ ]:
pca = PCA(pe, x, y)

In [ ]:
plt.figure(figsize=(8,8))
pca.plot(scaling=2)
plt.axis('equal')

In [ ]:
pca_opt(pca, scaling=2)

With the monomials:

In [ ]:
ncurves, npoints = x.shape
cl = (npoints-1)*np.ones((1,ncurves),dtype=int)
u = representer(x,y,ncurves,cl,npoints,10)
u1 = np.zeros((ncurves,np.shape(u)[1]*np.shape(u)[2]*2))
for i in range(ncurves):
    u1[i,:] = np.matrix.flatten(u[i,:,:,:])

In [ ]:
pca = PCA(u1, x, y)

In [ ]:
pca.plot(scaling=.01)

In [ ]:
x, y = load_data('currents_data_200_500.mat', 32)

In [ ]:
pe = monomial_current_pca(x, y, 10)
pca = PCA(pe, x, y)
plt.figure(figsize=(8,8))
pca.plot(scaling=20)
plt.axis('equal');

In [ ]:
pca_opt(pca, scaling=20)

In [ ]:
curves = np.stack([x,y], axis=-1)
currents = [Current(space, curve) for curve in curves]

In [ ]:
pe = fem_current_pca(currents, x, y)

In [ ]:
pca = PCA(pe, x, y)

In [ ]:
pca.plot(scaling=6)

In [ ]:
pca_opt(pca, scaling=8)

d(0,1), d(0,2), d(0,3)
order 1, mesh = 10, 20, 40
(2.2949783713748606, 2.4006275209345507, 2.2316118057013319)
(1.6047046605854045, 1.6207933275335604, 1.6825498540372308)
(1.2664142928892494, 1.2783837142622918, 1.3259963291001282)

order 4, mesh = 5, 10, 20
(2.2997301392640863, 2.2460204102017722, 2.2746276352501957)
(1.7498901419675774, 1.7855146077354773, 1.7592703776709591)

d(0,1), d(0,2), d(1,2)
order 4, mesh = 5, 10, 20
(2.7899967862469111, 2.8270892665201925, 0.8246409669811765)
(2.8852668644254802, 2.9246305575121454, 1.0339455136119406)
(2.9389698376057898, 2.9713066427066308, 1.1315582359980916)